In [1]:
pip install wrds pandas pyarrow psycopg2-binary

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   ------------------------------------ --- 1.0/1.2 MB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import wrds
conn = wrds.Connection()  # 首次会弹出WRDS用户名密码

Enter your WRDS username [zshen]: sxz960
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


pgpass file created at C:\Users\zshen\AppData\Roaming\postgresql\pgpass.conf
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [11]:
preview = conn.raw_sql("""
    select gvkey, conm, sic, fic, costat
    from comp.company
    where fic = 'USA'
    order by gvkey
    limit 5;
""")
preview

,gvkey,conm,sic,fic,costat
0,001000,A & E PLASTIK PAK INC,3089,USA,I
1,001001,A & M FOOD SERVICES INC,5812,USA,I
2,001002,AAI CORP,3825,USA,I
3,001003,A.A. IMPORTING CO INC,5712,USA,I
4,001004,AAR CORP,5080,USA,A


In [13]:
cols_check = conn.raw_sql("select * from comp.company limit 5;")
cols_check.columns.tolist()

['conm',
 'gvkey',
 'add1',
 'add2',
 'add3',
 'add4',
 'addzip',
 'busdesc',
 'cik',
 'city',
 'conml',
 'costat',
 'county',
 'dlrsn',
 'ein',
 'fax',
 'fic',
 'fyrc',
 'ggroup',
 'gind',
 'gsector',
 'gsubind',
 'idbflag',
 'incorp',
 'loc',
 'naics',
 'phone',
 'prican',
 'prirow',
 'priusa',
 'sic',
 'spcindcd',
 'spcseccd',
 'spcsrc',
 'state',
 'stko',
 'weburl',
 'dldte',
 'ipodate',
 'curr_sp500_flag']

In [29]:
try:
    conn.close()
except:
    pass

import wrds
conn = wrds.Connection()  # 重新登录

Enter your WRDS username [zshen]: sxz960
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


pgpass file created at C:\Users\zshen\AppData\Roaming\postgresql\pgpass.conf
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [31]:
# 先看 comp.security，有些学校实例是 comp.secm（两者字段很像）
sec_cols = conn.raw_sql("select * from comp.security limit 5;")
sec_cols.columns.tolist()

['tic',
 'gvkey',
 'iid',
 'cusip',
 'dlrsni',
 'dsci',
 'epf',
 'exchg',
 'excntry',
 'ibtic',
 'isin',
 'secstat',
 'sedol',
 'tpci',
 'dldtei',
 'curr_sp500_flag']

In [33]:
skeleton = conn.raw_sql("""
    select c.gvkey, c.conm, c.fic, c.sic, c.costat,
           s.tic, s.exchg, s.cusip
    from comp.company as c
    left join comp.security as s
        on c.gvkey = s.gvkey
    where c.fic = 'USA'
    order by c.gvkey
    limit 10;
""")
skeleton

,gvkey,conm,fic,sic,costat,tic,exchg,cusip
0,001000,A & E PLASTIK PAK INC,USA,3089,I,AE.2,12,000032102
1,001001,A & M FOOD SERVICES INC,USA,5812,I,AMFD.,14,000165100
2,001002,AAI CORP,USA,3825,I,AAIC.1,13,000352104
3,001003,A.A. IMPORTING CO INC,USA,5712,I,ANTQ,19,000354100
4,001004,AAR CORP,USA,5080,A,AIR,11,000361105
5,001005,A.B.A. INDUSTRIES INC,USA,3724,I,ABA.2,12,000370106
6,001006,ABC INDS INC,USA,2711,I,1145B,1,000736108
7,001007,ABKCO INDUSTRIES INC,USA,3652,I,4135B,1,000774109
8,001008,ABM COMPUTER SYSTEMS INC,USA,3577,I,ABMC.,13,000775106
9,001009,ABS INDUSTRIES INC,USA,3460,I,ABSI.1,19,000781104


In [35]:
# 1) 美国公司基础信息（company）
company_basic = conn.raw_sql("""
    SELECT gvkey, conm, fic, sic, costat
    FROM comp.company
    WHERE fic = 'USA'
""")

# 2) security 里挑主证券：iid='01'
sec_primary = conn.raw_sql("""
    SELECT gvkey, iid, tic, exchg, cusip
    FROM comp.security
    WHERE iid = '01'
""")

# 3) 合并：一行一公司（可能仍有极少数gvkey在security没有iid='01'，tic会是空）
master = (company_basic
          .merge(sec_primary, on='gvkey', how='left')
          .drop_duplicates()
          .reset_index(drop=True))

print("唯一公司数(gvkey)：", master['gvkey'].nunique(), "| 行数：", len(master))
master.head(10)

唯一公司数(gvkey)： 41244 | 行数： 41244


,gvkey,conm,fic,sic,costat,iid,tic,exchg,cusip
0,001000,A & E PLASTIK PAK INC,USA,3089,I,01,AE.2,12,000032102
1,001001,A & M FOOD SERVICES INC,USA,5812,I,01,AMFD.,14,000165100
2,001002,AAI CORP,USA,3825,I,01,AAIC.1,13,000352104
3,001003,A.A. IMPORTING CO INC,USA,5712,I,01,ANTQ,19,000354100
4,001004,AAR CORP,USA,5080,A,01,AIR,11,000361105
5,001005,A.B.A. INDUSTRIES INC,USA,3724,I,01,ABA.2,12,000370106
6,001006,ABC INDS INC,USA,2711,I,01,1145B,1,000736108
7,001007,ABKCO INDUSTRIES INC,USA,3652,I,01,4135B,1,000774109
8,001008,ABM COMPUTER SYSTEMS INC,USA,3577,I,01,ABMC.,13,000775106
9,001009,ABS INDUSTRIES INC,USA,3460,I,01,ABSI.1,19,000781104


In [37]:
funda_preview = conn.raw_sql("""
    SELECT gvkey, datadate, fyear, fyr, at, lt, sale
    FROM comp.funda
    WHERE fic = 'USA'
      AND indfmt = 'INDL'
      AND datafmt = 'STD'
      AND consol = 'C'
      AND datadate BETWEEN '2000-01-01' AND '2025-12-31'
    LIMIT 20;
""")
funda_preview.head()

,gvkey,datadate,fyear,fyr,at,lt,sale
0,001004,2000-05-31,1999,5,740.998,401.483,1024.333
1,001004,2001-05-31,2000,5,701.854,361.642,874.255
2,001004,2002-05-31,2001,5,710.199,399.964,638.721
3,001004,2003-05-31,2002,5,686.621,391.633,606.337
4,001004,2004-05-31,2003,5,709.292,407.608,651.958


In [39]:
# 取 2000–2005 年，美国、标准口径、合并报表、工业口径
funda_0005 = conn.raw_sql("""
    SELECT gvkey, datadate, fyear, fyr, at, lt, sale
    FROM comp.funda
    WHERE fic = 'USA'
      AND indfmt = 'INDL'
      AND datafmt = 'STD'
      AND consol = 'C'
      AND datadate BETWEEN '2000-01-01' AND '2005-12-31'
""")
len(funda_0005), funda_0005.head()

(54001,
     gvkey    datadate  fyear  fyr       at       lt      sale
 0  001004  2000-05-31   1999    5  740.998  401.483  1024.333
 1  001004  2001-05-31   2000    5  701.854  361.642   874.255
 2  001004  2002-05-31   2001    5  710.199  399.964   638.721
 3  001004  2003-05-31   2002    5  686.621  391.633   606.337
 4  001004  2004-05-31   2003    5  709.292  407.608   651.958)

In [41]:
cols_master = ['gvkey','conm','fic','sic','costat','tic','exchg','cusip']
panel_0005 = (funda_0005
              .merge(master[cols_master], on='gvkey', how='left')
              .drop_duplicates()
              .reset_index(drop=True))

print("联表后行数：", len(panel_0005), 
      " | 覆盖公司数：", panel_0005['gvkey'].nunique())
panel_0005.head()

联表后行数： 54001  | 覆盖公司数： 12006


,gvkey,datadate,fyear,fyr,at,lt,sale,conm,fic,sic,costat,tic,exchg,cusip
0,001004,2000-05-31,1999,5,740.998,401.483,1024.333,AAR CORP,USA,5080,A,AIR,11,000361105
1,001004,2001-05-31,2000,5,701.854,361.642,874.255,AAR CORP,USA,5080,A,AIR,11,000361105
2,001004,2002-05-31,2001,5,710.199,399.964,638.721,AAR CORP,USA,5080,A,AIR,11,000361105
3,001004,2003-05-31,2002,5,686.621,391.633,606.337,AAR CORP,USA,5080,A,AIR,11,000361105
4,001004,2004-05-31,2003,5,709.292,407.608,651.958,AAR CORP,USA,5080,A,AIR,11,000361105


In [43]:
ordered = ['gvkey','conm','tic','exchg','cusip','fic','sic','costat',
           'fyear','fyr','datadate','at','lt','sale']
panel_0005 = panel_0005[ordered]

panel_0005.to_csv("panel_USA_2000_2005.csv", index=False)
print("Saved: panel_USA_2000_2005.csv")
panel_0005.head(10)

Saved: panel_USA_2000_2005.csv


,gvkey,conm,tic,exchg,cusip,fic,sic,costat,fyear,fyr,datadate,at,lt,sale
0,001004,AAR CORP,AIR,11,000361105,USA,5080,A,1999,5,2000-05-31,740.998,401.483,1024.333
1,001004,AAR CORP,AIR,11,000361105,USA,5080,A,2000,5,2001-05-31,701.854,361.642,874.255
2,001004,AAR CORP,AIR,11,000361105,USA,5080,A,2001,5,2002-05-31,710.199,399.964,638.721
3,001004,AAR CORP,AIR,11,000361105,USA,5080,A,2002,5,2003-05-31,686.621,391.633,606.337
4,001004,AAR CORP,AIR,11,000361105,USA,5080,A,2003,5,2004-05-31,709.292,407.608,651.958
5,001004,AAR CORP,AIR,11,000361105,USA,5080,A,2004,5,2005-05-31,732.23,417.486,747.848
6,001010,ACF INDUSTRIES INC,4165A,0,00099V004,USA,3743,I,2000,12,2000-12-31,3794.5,2794.5,443.8
7,001010,ACF INDUSTRIES INC,4165A,0,00099V004,USA,3743,I,2001,12,2001-12-31,3723.1,2694.5,366.5
8,001010,ACF INDUSTRIES INC,4165A,0,00099V004,USA,3743,I,2002,12,2002-12-31,3702.5,2735.1,313.3
9,001010,ACF INDUSTRIES INC,4165A,0,00099V004,USA,3743,I,2003,12,2003-12-31,4832.1,3084.3,2339.1


In [51]:
os.makedirs("out/funda_panel", exist_ok=True)

def fetch_funda_year(year):
    sql = f"""
        SELECT gvkey, datadate, fyear, fyr, at, lt, sale
        FROM comp.funda
        WHERE fic = 'USA'
          AND indfmt = 'INDL'
          AND datafmt = 'STD'
          AND consol = 'C'
          AND fyear = {year}
    """
    dfy = conn.raw_sql(sql)
    
    cols_master = ['gvkey','conm','fic','sic','costat','tic','exchg','cusip']
    panel_y = (master[cols_master].merge(dfy, on='gvkey', how='left')
                              .drop_duplicates()
                              .reset_index(drop=True))
    
    ordered = ['gvkey','conm','tic','exchg','cusip','fic','sic','costat',
               'fyear','fyr','datadate','at','lt','sale']
    panel_y = panel_y.reindex(columns=ordered)
    
    out_path = f"out/funda_panel/panel_USA_{year}.parquet"
    panel_y.to_parquet(out_path, index=False)
    print(year, len(panel_y), "rows ->", out_path)
    return out_path

In [53]:
_ = fetch_funda_year(2000)
_ = fetch_funda_year(2001)

2000 41244 rows -> out/funda_panel/panel_USA_2000.parquet
2001 41244 rows -> out/funda_panel/panel_USA_2001.parquet


In [55]:
for yr in range(2000, 2025):   # 年报通常更新到 2024
    _ = fetch_funda_year(yr)

2000 41244 rows -> out/funda_panel/panel_USA_2000.parquet
2001 41244 rows -> out/funda_panel/panel_USA_2001.parquet
2002 41244 rows -> out/funda_panel/panel_USA_2002.parquet
2003 41244 rows -> out/funda_panel/panel_USA_2003.parquet
2004 41244 rows -> out/funda_panel/panel_USA_2004.parquet
2005 41244 rows -> out/funda_panel/panel_USA_2005.parquet
2006 41244 rows -> out/funda_panel/panel_USA_2006.parquet
2007 41244 rows -> out/funda_panel/panel_USA_2007.parquet
2008 41244 rows -> out/funda_panel/panel_USA_2008.parquet
2009 41244 rows -> out/funda_panel/panel_USA_2009.parquet
2010 41245 rows -> out/funda_panel/panel_USA_2010.parquet
2011 41245 rows -> out/funda_panel/panel_USA_2011.parquet
2012 41245 rows -> out/funda_panel/panel_USA_2012.parquet
2013 41245 rows -> out/funda_panel/panel_USA_2013.parquet
2014 41245 rows -> out/funda_panel/panel_USA_2014.parquet
2015 41245 rows -> out/funda_panel/panel_USA_2015.parquet
2016 41245 rows -> out/funda_panel/panel_USA_2016.parquet
2017 41245 row

In [57]:
import pandas as pd

df2000 = pd.read_parquet("out/funda_panel/panel_USA_2000.parquet")
print("列数量:", len(df2000.columns))
print("列名列表:")
print(df2000.columns.tolist())

列数量: 14
列名列表:
['gvkey', 'conm', 'tic', 'exchg', 'cusip', 'fic', 'sic', 'costat', 'fyear', 'fyr', 'datadate', 'at', 'lt', 'sale']
